In [8]:
#custom condition for WebdriverWait to check if all elementes are in the filtered city
class check_all_elements_present(object):
    def __init__(self, locator, city):
        self.locator = locator
        self.city = city

    def __call__(self, driver):
        elements = driver.find_elements(*self.locator)
        #if all elements are in the filtered city, return True
        if all(element.text == self.city for element in elements):
            return elements
        else:
            return False

In [57]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd



#custom condition for WebdriverWait to check if all elementes are in the filtered city
# class check_all_elements_present(object):
#     def __init__(self, locator, city):
#         self.locator = locator
#         self.city = city

#     def __call__(self, driver):
#         elements = driver.find_elements(*self.locator)
#         #if all elements are in the filtered city, return True
#         if all(element.text == self.city for element in elements):
#             return elements
#         else:
#             return False
    
def convert_to_int(x):   
    return 0 if x == 'No Tiene' or x == '' else x

def get_property_details(driver, href):
    driver.get(href)
    # print('-'*50)
    # print(href)
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="gatsby-focus-wrapper"]/div/main/section/article/section/div/ul/li[2]/div/div[2]/div/div/div/div/section/article/ul/li[1]/ul'))
        )
        # print('Success')
        # num_success += 1
    except Exception as e:
        # print(f'Error: {e}')
        print(f'Error in {href}')
        # num_errors += 1
        return None

    property_soup = BeautifulSoup(driver.page_source, 'html.parser')
    details = property_soup.find_all('div', class_='detail-item')

    details_to_add = {'conjunto': '', 'administración': '', 'estrato': '', 'antiguedad': '', 'remodelado': '',
                      'área': '', 'habitaciones': '', 'baños': '', 'garajes': '', 'elevadores': '', 'tipo_de_inmueble': '',
                      'deposito': '', 'porteria': '', 'zona_de_lavanderia': '', 'gas': '', 'parqueadero': '',
                      'precio': '', 'direccion': '', 'nombre': '', 'descripcion': ''}
    
    for detail in details:
        detail_title = detail.find('p', class_='item-title').text.lower().replace(' ', '_').replace('*', '')
        detail_description = detail.find('p', class_='item-description').text
        details_to_add[detail_title] = detail_description

    details_to_add.update({
        'precio': property_soup.find('p', class_='current-price').text,
        'direccion': property_soup.find('div', class_='title-location').text,
        'nombre': property_soup.find('h1', class_='header-title').text,
        'descripcion': property_soup.find('p', class_='description-text').text
    })

    return details_to_add

def scrape_page(driver, city, page):
    driver.get(f'https://habi.co/venta-apartamentos/{city}?page={page}')
    # WebDriverWait(driver, 10).until(check_all_elements_present((By.CLASS_NAME, 'card-city'), 'BOGOTÁ'))
    time.sleep(7)
        
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    titles = soup.find_all('a', class_='card-details')
    hrefs = [title.get('href') for title in titles]

    properties_data = [get_property_details(driver, href) for href in hrefs if href] #filter out None values
    properties_data = [property for property in properties_data if property] #filter none values from properties_data
    # num_errors = len([property for property in properties_data if property is None])
    properties_df = pd.DataFrame(properties_data)


    # print(f'Page {page} scraped with {num_errors} errors')
    return properties_df

city = 'bogota'
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
num_pages_to_scrape = 19
properties_df = pd.DataFrame()
# total_errors = 0
for page in range(1, num_pages_to_scrape + 1):
    properties_df= pd.concat([properties_df, scrape_page(driver, city, page)])
    print(f'Page {page} scraped')
driver.quit()

#cleaning process of the dataframe
properties_df['administración'] = properties_df['administración'].apply(lambda x: '0' if x == '' else x.replace('$', '').replace('.', '').strip()).astype('int')
properties_df['elevadores'] = properties_df['elevadores'].apply(convert_to_int).astype('int')
properties_df['área'] = properties_df['área'].apply(lambda x: x.replace('m2', '').replace(',', '.')).astype('float')
properties_df['habitaciones'] = properties_df['habitaciones'].apply(convert_to_int).astype('int')
properties_df['baños'] = properties_df['baños'].apply(convert_to_int).astype('int')
properties_df['garajes'] = properties_df['garajes'].apply(convert_to_int).astype('int')
properties_df['antiguedad'] = properties_df['antiguedad'].str.extract(r'(\d+)').astype('int')
properties_df['deposito'] = properties_df['deposito'].apply(convert_to_int).astype('int')
properties_df['precio'] = properties_df['precio'].str.replace('.','').str.extract(r'(\d+)').astype('float')
properties_df['barrio'] = properties_df['direccion'].apply(lambda x: x.split('/')[1].strip())
properties_df['direccion'] = properties_df['direccion'].apply(lambda x: x.split('/')[0].strip())
properties_df['nombre'] = properties_df['nombre'].apply(lambda x: x.split('Bogotá')[1].strip())


#saving the dataframe to a csv file
properties_df.to_csv('properties.csv', index=False)
print('Scrapping finished')

Error in https://habi.co/venta-apartamentos/13754786935/portal-iberia---apartamento-venta-prado-veraniego-norte-suba
Error in https://habi.co/venta-apartamentos/8739930694/edificio-alcazar-novara-apartamento-venta-cedritos-usaquen
Page 1 scraped
Error in https://habi.co/venta-apartamentos/13770740830/arrayanes-suba---apartament-venta-tuna-baja-suba
Error in https://habi.co/venta-apartamentos/8151147319/rincon-ipanema-apartamento-venta-osorio-iii-kennedy
Page 2 scraped
Error in https://habi.co/venta-apartamentos/12899029518/edificio-villorio-apartamento-venta-chapinero-norte
Error in https://habi.co/venta-apartamentos/9483837367/quintas-santa-rita-4-casa-venta-tibabuyes-2-suba
Error in https://habi.co/venta-apartamentos/14797560916/apartamento-venta-pinar-novalena-arborizadora-madelena
Error in https://habi.co/venta-apartamentos/7527552138/nuevo-recreo-1-apartamento-venta-ciudadela-recreo-bosa
Error in https://habi.co/venta-apartamentos/15327751090/balcones-colina-apartamento-venta-gilm

In [58]:
properties_df

,conjunto,administración,estrato,antiguedad,remodelado,área,habitaciones,baños,garajes,elevadores,...,deposito,porteria,zona_de_lavanderia,gas,parqueadero,precio,direccion,nombre,descripcion,barrio
0,Santa Mónica,532000,4,37,Si,86.0,1,1,1,1,...,0,24 hrs,No Tiene,Si,No,313900000.0,Avenida Carrera 3 # 59-65,Santa Mónica,"Apartamento en venta de 86 m2, con vista exter...",PARDO RUBIO
1,Chicó Milano 101,0,6,7,Si,77.0,1,2,2,1,...,1,24 hrs,No Tiene,Si,Si,440100000.0,Carrera 12 #101A-18,Chicó Milano 101,"Apartamento en venta de 77m2, con vista interi...",USAQUEN
2,Portal del Belmira,811893,4,14,Si,109.0,3,4,2,1,...,0,24 hrs,No Tiene,Si,Si,495000000.0,Calle 146 #7F-54,Portal del Belmira,"Apartamento en venta de 109 m2, con vista Inte...",LOS CEDROS
3,Carmel Reservado,400200,4,11,Si,76.0,3,2,1,2,...,0,24 hrs,No Tiene,Si,Si,442300000.0,KR 54 152A 35,Carmel Reservado,"Apartamento en venta de 76m2, con vista exteri...",EL PRADO
4,Toscana 1,270000,4,20,Si,105.0,4,2,1,0,...,1,24 hrs,No Tiene,Si,Si,387000000.0,CL 168 14 55,Toscana 1,"Casa en venta de 105m2, con vista exterior, ub...",TOBERIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,Quintas de la Sabana 2,155000,2,10,No,57.0,2,1,0,0,...,0,24 hrs,No Tiene,Si,Si,190000000.0,Diagonal 77 # 123 a - 88,Quintas de la Sabana 2,"Casa en venta de 57m2, con vista exterior, (ca...",GARCES NAVAS
27,Tekto San Marcos,0,4,6,No,51.0,1,2,1,1,...,0,24 hrs,Comunal,Si,Si,366000000.0,Carrera 18 # 40b-74,Tekto San Marcos,"Apartamento en venta de 51m2, con vista interi...",CIUDAD MONTES
28,Edificio Riviera 2,55000,2,17,No,50.0,3,1,0,0,...,0,No,No Tiene,Si,No,127000000.0,Carrera 81a#58g-24sur,Edificio Riviera 2,"Apartamento en venta de 50 m2, con vista inter...",BOSA CENTRAL
29,El Almendro,816600,6,32,No,112.0,3,2,2,1,...,1,24 hrs,No Tiene,Si,Si,500000000.0,Cra 13A # 102-16,El Almendro,"Apartamento en venta de 112m2, con vista exter...",USAQUEN
